In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
import os

In [5]:
today_str = dt.date.today().strftime("%Y_%m_%d")
os.getcwd()

'/Users/corybaird/Desktop/Test'

In [8]:
api_file = open('api_bea.txt')
api= "".join(api_file.readlines())

# BEA Regional data (GDP & Personal Income)


# 1. [BEA API](https://apps.bea.gov/api/_pdf/bea_web_service_api_user_guide.pdf)

## 1.1 Find parameters for entire data set

In [9]:
def param_request(method, dataset, param_name, format_type):
    api = api_code
    base = 'http://apps.bea.gov/api/data?&UserID='
    base_api = base+api

    method = '&method='+ method
    data_set = '&datasetname='+ dataset
    param = '&ParameterName='+param_name
    result_format = '&ResultFormat='+ format_type

    final_api = base_api + method + data_set + param + result_format

    request = requests.get(final_api)
    bs = BeautifulSoup(request.text, 'xml')
    
    return bs
    

In [10]:
linecodes = param_request(method = "GetParameterValues", 
                      dataset = 'Regional', 
                      param_name = 'LineCode',
                      format_type = 'XML')

In [11]:
#print(linecodes.prettify())

## 1.2 Find parameter codes for given table

## GetParameterValuesFiltered

In [12]:
def param_request_filter(method, dataset, param_name, table_name, format_type):
    api = '56CAFC0F-BA67-4159-9CD6-39D7860CA778'
    base = 'http://apps.bea.gov/api/data?&UserID='
    base_api = base+api

    method = '&method='+ method
    
    data_set = '&datasetname='+ dataset
    
    param = '&TargetParameter='+param_name
    
    table = '&TableName=' + table_name 
    
    result_format = '&ResultFormat='+ format_type

    final_api = base_api + method + data_set + param +table + result_format
    request = requests.get(final_api)
    bs = BeautifulSoup(request.text, 'xml')
    
    return bs
    

In [13]:
linecodes_table = param_request_filter(method = "GetParameterValuesFiltered", 
                      dataset = 'Regional', 
                      param_name = 'LineCode', #GEOFIPS
                      table_name = 'CAINC4',
                      format_type = 'XML')

In [14]:
print(linecodes_table.prettify())

<?xml version="1.0" encoding="utf-8"?>
<BEAAPI>
 <Request>
  <RequestParam ParameterName="METHOD" ParameterValue="GETPARAMETERVALUESFILTERED"/>
  <RequestParam ParameterName="RESULTFORMAT" ParameterValue="XML"/>
  <RequestParam ParameterName="TABLENAME" ParameterValue="CAINC4"/>
  <RequestParam ParameterName="USERID" ParameterValue="56CAFC0F-BA67-4159-9CD6-39D7860CA778"/>
  <RequestParam ParameterName="DATASETNAME" ParameterValue="REGIONAL"/>
  <RequestParam ParameterName="TARGETPARAMETER" ParameterValue="LINECODE"/>
 </Request>
 <Results>
  <ParamValue Desc="[CAINC4] Personal income" Key="10"/>
  <ParamValue Desc="[CAINC4] Nonfarm personal income" Key="11"/>
  <ParamValue Desc="[CAINC4] Farm income" Key="12"/>
  <ParamValue Desc="[CAINC4] Population" Key="20"/>
  <ParamValue Desc="[CAINC4] Per capita personal income" Key="30"/>
  <ParamValue Desc="[CAINC4] Earnings by place of work" Key="35"/>
  <ParamValue Desc="[CAINC4] Contributions for government social insurance" Key="36"/>
  <Pa

### 1.3 Download data

In [15]:
def target_url(method, dataset, table_name, linecode, frequency, geofips, year, format):
    api = '56CAFC0F-BA67-4159-9CD6-39D7860CA778'
    base = 'http://apps.bea.gov/api/data?&UserID='
    base_api = base+api

    method = '&method='+ method
    data_set = '&datasetname='+ dataset
    
    line_geo_year = '&GeoFIPS='+geofips+'&LineCode='+linecode+'&Year='+year
    
    #line_geo_year = '&GeoFIPS='+ geofips+ '&Year='+year
    
    freq = 'Frequency=' + frequency
    
    table = '&TableName='+ table_name
    result_format = '&ResultFormat='+ format

    final_api = base_api + method + data_set + table + line_geo_year + result_format

    request = requests.get(final_api)
    bs = BeautifulSoup(request.text, 'xml')
    return bs
    

# 2. Download data

## 2.1 Regional GDP

In [16]:
regional = target_url(method = "GetData", 
                      dataset = 'Regional', 
                      table_name = 'CAINC4', 
                      geofips = 'COUNTY', 
                      linecode = '10',
                      frequency = 'Q',
                      year = 'All', 
                      format = 'XML')


## 2.2 Show full data

In [17]:
#print(regional.prettify())

In [18]:
#regional

## 2.3 Only data

In [19]:
for code in regional.find_all('Data'):
    print(code)
    break

<Data CL_UNIT="Thousands of dollars" Code="CAINC4-10" DataValue="424,990" GeoFips="01001" GeoName="Autauga, AL" TimePeriod="1987" UNIT_MULT="3"/>


### 2.3.1 Parse data

In [20]:
values, date, state = [],[],[]
for row in regional.find_all('Data'):  
    values.append(row.get('DataValue'))
    date.append(row.get('TimePeriod'))
    state.append(row.get('GeoName'))
data_dict = {'value':values, 'date':date, 'state':state}

## 2.3.2 Convert to dataframe

In [21]:
df = pd.DataFrame.from_dict(data_dict)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163280 entries, 0 to 163279
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   value   163280 non-null  object
 1   date    163280 non-null  object
 2   state   163280 non-null  object
dtypes: object(3)
memory usage: 3.7+ MB


In [22]:
df.head(3)

,value,date,state
0,"424,990",1987,"Autauga, AL"
1,"503,406",1989,"Autauga, AL"
2,"869,142",1997,"Autauga, AL"


### 2.3.2.1 Convert to datatime and float

In [23]:
df = df[df.state.str.contains('Los Angeles')]
df.value = [float(x.replace(',','').replace("(NA)",'')) for x in df.value]
df.date = [pd.to_datetime(x) for x in df.date]
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 11960 to 12011
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   value   52 non-null     float64       
 1   date    52 non-null     datetime64[ns]
 2   state   52 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1.6+ KB
None


In [27]:
df.state.value_counts()[:3]

Los Angeles, CA    52
Name: state, dtype: int64

In [28]:
df.head(3)

,value,date,state
11960,51826029.0,1975-01-01,"Los Angeles, CA"
11961,237616263.0,1997-01-01,"Los Angeles, CA"
11962,114723500.0,1983-01-01,"Los Angeles, CA"
